### 爬蟲

In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup


options = Options()
options.add_argument("--disable-notifications")

chrome = webdriver.Chrome('./chromedriver', chrome_options=options)
chrome.get("https://www.wantgoo.com/stock/dividend-yield")

for x in range(1, 4):
    chrome.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    time.sleep(5)

soup = BeautifulSoup(chrome.page_source, 'html.parser')

# stock name
titles = soup.find_all('td', {
    'class': 'zw'})

titlelist = []

for title in titles:
    titlelist.append(title.text)


# stock prices
prices = soup.find_all('td', {
    "c-model-dazzle": "text:close,class:upDn"})

pricelist = []

for price in prices:
    if price.text == None:
        pricelist.append("none")
    else:
        pricelist.append(price.text)

# dividend yield
dividends = soup.find_all('td', {
    "c-model": r"`{nearby4QuartersYield}%`"})

dividendlist = []

for dividend in dividends:
    dividendlist.append(dividend.text)


# cashdividend
cashdividends = soup.find_all('td', {
    "c-model": "cashDividend"})

cashdividendlist = []

for cashdividend in cashdividends:
    cashdividendlist.append(cashdividend.text)

# stocknumber
numbers = soup.find_all('a', {
    "c-model-dazzle":"text:id,href:url"})

numberlist = []

for number in numbers:
    numberlist.append(number.text)

chrome.quit()

# list to dataframe
all_list = [titlelist, pricelist, dividendlist, cashdividendlist,numberlist]

df = pd.DataFrame(all_list).transpose()
df.columns = ["stockname", "price", "dividend-field", "cash-dividend","number"]
print(df)
df.to_csv('results.csv')


    stockname  price dividend-field cash-dividend number
0          陽明   61.1         32.73%            20   2609
1          富旺  11.25         30.41%          0.25   6219
2          申豐   57.8         25.95%            15   6582
3          群利   13.8         25.28%      3.488134   7558
4          紘康   53.2         20.47%         10.89   6457
..        ...    ...            ...           ...    ...
196        惠光  37.45          7.97%      2.986371   6508
197        茂綸   42.1          7.96%          3.35   6227
198       國泰金  44.05          7.95%           3.5   2882
199       永豐實   37.8          7.94%             3   6790
200                                                     

[201 rows x 5 columns]


### 取資料

In [4]:
import pandas as pd
data = pd.read_csv('results.csv')
data

,Unnamed: 0,stockname,price,dividend-field,cash-dividend,number
0,0,陽明,61.10,32.73%,20.000000,2609
1,1,富旺,11.25,30.41%,0.250000,6219
2,2,申豐,57.80,25.95%,15.000000,6582
3,3,群利,13.80,25.28%,3.488134,7558
4,4,紘康,53.20,20.47%,10.890000,6457
...,...,...,...,...,...,...
196,196,惠光,37.45,7.97%,2.986371,6508
197,197,茂綸,42.10,7.96%,3.350000,6227
198,198,國泰金,44.05,7.95%,3.500000,2882
199,199,永豐實,37.80,7.94%,3.000000,6790


In [5]:
data.iloc[:10]

,Unnamed: 0,stockname,price,dividend-field,cash-dividend,number
0,0,陽明,61.10,32.73%,20.000000,2609
1,1,富旺,11.25,30.41%,0.250000,6219
2,2,申豐,57.80,25.95%,15.000000,6582
3,3,群利,13.80,25.28%,3.488134,7558
4,4,紘康,53.20,20.47%,10.890000,6457
5,5,陸海,20.65,19.37%,4.000000,5603
6,6,敦泰,82.90,19.1%,15.836300,3545
7,7,世豐,59.30,17.93%,3.997866,2065
8,8,宏遠證,10.05,17.91%,1.800000,6015
9,9,萬海,58.70,17.89%,10.500000,2615


### 獲利計算

In [6]:
money = int(input("你要投資多少錢"))


topstock = data.iloc[:10]
topstock["profit"] = money/topstock["price"] * topstock["cash-dividend"]

print("------------------------------------------------------------------")
print("推薦這幾間公司給你")
print(topstock[["stockname","price","profit"]])

------------------------------------------------------------------
推薦這幾間公司給你
  stockname  price      profit
0        陽明  61.10  327.332242
1        富旺  11.25   22.222222
2        申豐  57.80  259.515571
3        群利  13.80  252.763333
4        紘康  53.20  204.699248
5        陸海  20.65  193.704600
6        敦泰  82.90  191.028951
7        世豐  59.30   67.417639
8       宏遠證  10.05  179.104478
9        萬海  58.70  178.875639


C:\Users\user\AppData\Local\Temp\ipykernel_23624\4134720341.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  topstock["profit"] = money/topstock["price"] * topstock["cash-dividend"]


In [19]:
topstock

,Unnamed: 0,stockname,price,dividend-field,cash-dividend,number,profit
0,0,陽明,61.10,32.73%,20.000000,2609,327.332242
1,1,富旺,11.25,30.41%,0.250000,6219,22.222222
2,2,申豐,57.80,25.95%,15.000000,6582,259.515571
3,3,群利,13.80,25.28%,3.488134,7558,252.763333
4,4,紘康,53.20,20.47%,10.890000,6457,204.699248
5,5,陸海,20.65,19.37%,4.000000,5603,193.704600
6,6,敦泰,82.90,19.1%,15.836300,3545,191.028951
7,7,世豐,59.30,17.93%,3.997866,2065,67.417639
8,8,宏遠證,10.05,17.91%,1.800000,6015,179.104478
9,9,萬海,58.70,17.89%,10.500000,2615,178.875639


In [34]:
comnum = topstock[topstock["stockname"] == "陽明"]["number"].astype("string")
comnum = [str(x) for x in comnum][0]
print(comnum)

0    2609
Name: number, dtype: string
2609


In [31]:
import yfinance as yf

stocknum = "2609.TW"
df = yf.Ticker(stocknum).history(period="10y")
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2013-05-27,21.765425,21.932850,21.597999,21.681711,2623795,0.0,0.0
2013-05-28,21.681712,21.765426,21.430572,21.514286,2042421,0.0,0.0
2013-05-29,21.514286,21.681712,21.514286,21.598000,1751734,0.0,0.0
2013-05-30,21.597999,21.597999,21.095719,21.179432,4187606,0.0,0.0
2013-05-31,21.430571,21.430571,21.263146,21.430571,2160884,0.0,0.0
...,...,...,...,...,...,...,...
2023-05-19,61.400002,61.599998,61.000000,61.299999,8611586,0.0,0.0
2023-05-22,61.400002,62.500000,61.099998,62.099998,15044999,0.0,0.0
2023-05-23,62.200001,62.200001,61.599998,61.700001,9943548,0.0,0.0


In [17]:
com = input("你要投資哪間公司")
comnum = str(topstock[topstock["stockname"] == com]["number"])
print(comnum)
print(type(comnum))


word = comnum+"tw"
print(word)

Series([], Name: number, dtype: object)
<class 'str'>
Series([], Name: number, dtype: object)tw
